In [1]:
from docx import Document
import pandas as pd
import tkinter.ttk as ttk
import tkinter.messagebox as msgbox
from tkinter import * # __all__
from tkinter import filedialog

OP_ROW = 1 
HEADER_ROW = 2 
FIRST_ROW = 3 


In [2]:
def getLeftHeaderCols(tb):
    op_cols = []
    ref_cols = []
    item_cols = []
    des_cols = []
    qty_cols = []
    for idx, _ in enumerate(tb.columns):
        if 'op' in tb.cell(OP_ROW, idx).text.lower().strip():
            op_cols.append(idx)
        if 'ref' in tb.cell(HEADER_ROW, idx).text.lower().strip():
            ref_cols.append(idx)
        if 'man.item' in tb.cell(HEADER_ROW, idx).text.lower().strip():
            item_cols.append(idx)
        if 'description' in tb.cell(HEADER_ROW, idx).text.lower().strip():
            des_cols.append(idx)
        if 'qty' in tb.cell(HEADER_ROW, idx).text.lower().strip():
            qty_cols.append(idx)
    if ref_cols:
        return {'ref':ref_cols[0],'item':item_cols[0],'des':des_cols[0],'qty':qty_cols[0]}
    
def getRightHeaderCols(tb):
    op_cols = []
    ref_cols = []
    item_cols = []
    des_cols = []
    qty_cols = []
    for idx, _ in enumerate(tb.columns):
        if 'op' in tb.cell(OP_ROW, idx).text.lower().strip():
            op_cols.append(idx)
        if 'ref' in tb.cell(HEADER_ROW, idx).text.lower().strip():
            ref_cols.append(idx)
        if 'man.item' in tb.cell(HEADER_ROW, idx).text.lower().strip():
            item_cols.append(idx)
        if 'description' in tb.cell(HEADER_ROW, idx).text.lower().strip():
            des_cols.append(idx)
        if 'qty' in tb.cell(HEADER_ROW, idx).text.lower().strip():
            qty_cols.append(idx)
            
    right_header_cols = {'ref': int ,'item':int,'des':int,'qty':int}
    try:
        right_header_cols['ref'] = ref_cols[2]
        for col in item_cols:
            if col > right_header_cols['ref']:
                right_header_cols['item'] = col
                break
        for col in des_cols:
            if col > right_header_cols['ref']:
                right_header_cols['des'] = col
                break
        for col in qty_cols:
            if col > right_header_cols['ref']:
                right_header_cols['qty'] = col
                break
    except:
        pass
    return right_header_cols

def GetLeftItemsList(table):
    item = []
    items = []
    headers = getLeftHeaderCols(table)
    for i,cells in enumerate(table.rows) : # left
        op = table.cell(OP_ROW, 3).text
        try:
            ref = table.cell(FIRST_ROW + i, headers['ref']).text.replace('\n','')
            item = table.cell(FIRST_ROW + i, headers['item']).text
            des = table.cell(FIRST_ROW  + i, headers['des']).text
            qty = table.cell(FIRST_ROW + i, headers['qty']).text
        except:
            continue
        if qty.isnumeric():
            item = [op, ref, item, des, qty]
            items.append(item)
    return items

def GetRightItemsList(table):
    item = []
    items = []
    headers = getRightHeaderCols(table)
    for i,cells in enumerate(table.rows) : # left
        op = table.cell(OP_ROW, 19).text
        try:
            ref = table.cell(FIRST_ROW + i, headers['ref']).text.replace('\n','')
            item = table.cell(FIRST_ROW + i, headers['item']).text
            des = table.cell(FIRST_ROW  + i, headers['des']).text
            qty = table.cell(FIRST_ROW + i, headers['qty']).text
        except:
            continue
        if qty.isnumeric():
            item = [op, ref, item, des, qty]
            items.append(item)
    return items

In [9]:
doc = Document('sop2.docx')  

for i, tb in enumerate(doc.tables):
    print(i, len(tb.rows), len(tb.columns))
    if len(tb.rows) > 10 : # 최소 열줄은 넘겨야...
        lefts = GetLeftItemsList(tb)
        rights = GetRightItemsList(tb)
    

0 4 15
1 4 13
2 15 13
3 5 13
4 4 13
5 8 19
6 19 34
7 17 28
8 17 28
9 15 28
10 14 27
11 17 31
12 19 28
13 14 27
14 17 31
15 20 28
16 14 27
17 15 31
18 16 31
19 15 29
20 13 27
21 16 31
22 20 30
23 14 28
24 17 29
25 16 30
26 17 30
27 13 29
28 16 30
29 16 30
30 14 29
31 16 30
32 16 30
33 16 30
34 16 30
35 17 29
36 16 29
37 16 30
38 17 31
39 16 29
40 17 29
41 17 29
42 15 31
43 14 28
44 15 31
45 17 30
46 17 30
47 17 30
48 16 30
49 16 29
50 16 29
51 16 29
52 17 29
53 15 29
54 14 29
55 16 29
56 16 29
57 16 29
58 8 28
59 8 28
60 17 29
61 16 29
62 8 28
63 12 29
64 14 29
65 16 27
66 16 29
67 16 30
68 16 29
69 18 30
70 17 30
71 16 29
72 17 30
73 17 30
74 17 31
75 15 30
76 12 28
77 15 29
78 17 29
79 17 29
80 15 30
81 16 29
82 16 29
83 16 29
84 17 29
85 3 10
86 3 10


In [ ]:
sopfile = ''

def add_file():
    global sopfile
    global file_name
    try:
        sopfile = filedialog.askopenfilename(initialdir=r'C:\Users\ParkGY\DocumentsCFLTYanadoo\DT Academy\SOPMeterial',title="select a file",
                                            filetypes =(("Word File","*.docx"),
                                            ("all files","*.*")))
    except:
        pass
    
def convert():       
    doc = Document(sopfile)
    
    df = pd.DataFrame(columns=['OperationStep','Ref.','Man.Item.No','Description','Qty'])
    for tb in doc.tables:
        if tb:
            lefts = GetLeftItemsList(tb)
            rights = GetRightItemsList(tb)
            for left in lefts:
                df.loc[len(df)] = left
            for right in rights:    
                df.loc[len(df)] = right
   
    f_name = sopfile.split('.')[0] + '.csv' 
    df.to_csv(f_name, encoding='utf-8-sig', index=False, mode='w', header=True)   


In [ ]:
root = Tk()
root.title("Docx SOP Converter")
root.geometry("300x150")

# Frame 
file_frame = Frame(root)
file_frame.pack(fill="x", padx=5, pady=5) # 간격 띄우기

btn_add_file = Button(file_frame, padx=5, pady=5, text="Open SOP", width=20, command=add_file)
btn_add_file.pack(side="top")

btn_start = Button(file_frame, padx=5, pady=5, text="Convert", width=20, command=convert)
btn_start.pack(side="top", padx=5, pady=5)
# Runs
root.mainloop()